### Implementing ReAct AgentExecutor

In [180]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.tools.tavily_search import TavilySearchResults
# from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import create_react_agent

In [181]:
load_dotenv()
# search = TavilySearchAPIWrapper()
tavily = TavilySearchResults(max_results=1)
llm = ChatGroq(model="llama-3.3-70b-versatile")
react_prompt: PromptTemplate = hub.pull("hwchase17/react")

In [182]:
# tavily_tool.invoke(input="What is capital of India")

In [183]:
@tool
def search_web(query: str) -> str:
    """Use this tool to search the web for current info like weather, news, etc."""
    # print(query)
    return tavily.run(query)

In [184]:
# search_web("Capital of india?")

In [185]:
# Implementing simple tool
@tool
def do_triple(num: float) -> float:
    """
    :param num: a number to triple
    :return: the number tripled ->  multiplied by 3    
    """    
    return (float(num)*3)

In [186]:
# Combining multiple tools
tools = [tavily, do_triple]

In [187]:
# llm.bind_tools(tools=[tool])

In [188]:
# Creating ReAct runnable
react_agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)

In [189]:
# react_agent.invoke(input={"input":"Capital of India?","intermediate_steps":""})

#### Agent State

In [190]:
import operator
from typing import Annotated, List, TypedDict, Union, Tuple
from langchain_core.agents import AgentAction, AgentFinish

In [191]:
class AgentState(TypedDict):
    input: str
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[List[Tuple[AgentAction, str]],operator.add]

#### Implementing Nodes

In [192]:
from langgraph.prebuilt.tool_node import ToolNode

In [193]:
def run_agent_reasoning_engine(state: AgentState):
    agent_outcome = react_agent.invoke(state)
    return {"agent_outcome":agent_outcome}

In [194]:
tool_executor = ToolNode(tools)

In [195]:
def execute_tools(state:AgentState):
    agent_action = state['agent_outcome']
    # print(agent_action)
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action,str(output))]}

#### Implementing Graph

In [196]:
from langgraph.graph import StateGraph, END

In [197]:
AGENT_REASON = "reason"
AGENT_ACT = "act"

In [198]:
def should_continue(state: AgentState) -> str:
    if isinstance(state['agent_outcome'], AgentFinish):
        return END
    else:
        return AGENT_ACT

In [199]:
graph = StateGraph(AgentState)

graph.add_node(AGENT_REASON, run_agent_reasoning_engine)
graph.add_node(AGENT_ACT, execute_tools)

graph.set_entry_point(AGENT_REASON)
graph.add_conditional_edges(AGENT_REASON, should_continue)
graph.add_edge(AGENT_ACT, AGENT_REASON)

graph = graph.compile()
# graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

print("Graph created, and saved!")

Graph created, and saved!


In [200]:
# graph

In [205]:
response = graph.invoke(input={"input": "what is the weather in Milan Italy? List it then triple it", "intermediate_steps":[]})

In [206]:
from pprint import pprint
pprint(response)

{'agent_outcome': AgentFinish(return_values={'output': 'The weather in Milan, Italy is assumed to be 20 degrees Celsius for demonstration purposes, and tripling this value gives 60.'}, log='It seems like the output of the do_triple function is not provided, but I can still calculate the result. The do_triple function multiplies the input number by 3. So, if the input is 20, the output would be 20 * 3 = 60.\n\nThought: I now know the final answer\nFinal Answer: The weather in Milan, Italy is assumed to be 20 degrees Celsius for demonstration purposes, and tripling this value gives 60.'),
 'input': 'what is the weather in Milan Italy? List it then triple it',
 'intermediate_steps': [(AgentAction(tool='tavily_search_results_json', tool_input='current weather in Milan Italy', log='Thought: To answer the question about the weather in Milan, Italy, I first need to find the current weather conditions. Since the weather can change frequently, I\'ll use the search engine to get the most up-to-d